In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

%matplotlib inline

In [ ]:
BATCH_SIZE = 128
IMAGE_SIZE = (128, 128)
EPOCHS = 2
LR = 1e-3

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/100-bird-species/285 birds/train",
    seed=42,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/100-bird-species/285 birds/valid",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/100-bird-species/285 birds/test",
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
class_names = train_ds.class_names
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[int(labels[i])])
        plt.axis("off")

## Model

In [ ]:
base_model = keras.applications.ResNet50(include_top=False)

In [ ]:
model = keras.Sequential()
model.add(keras.layers.InputLayer(IMAGE_SIZE + (3,)))
model.add(keras.layers.experimental.preprocessing.Rescaling(1./255))
model.add(base_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(500, activation="relu"))
model.add(keras.layers.Dense(len(class_names), activation="softmax"))

model.layers[1].trainable = False

model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=keras.optimizers.Adam(1e-3), 
              metrics=["accuracy"])


model.summary()

## Training

In [ ]:
train_ds = train_ds.prefetch(buffer_size=BATCH_SIZE)
valid_ds = valid_ds.prefetch(buffer_size=BATCH_SIZE)
model.fit(train_ds, epochs=1, validation_data=valid_ds)

In [ ]:
1 / len(class_names)

## Model 2

In [ ]:
model = keras.Sequential()
model.add(keras.layers.InputLayer(IMAGE_SIZE + (3,)))
model.add(keras.layers.experimental.preprocessing.Rescaling(1./255))
model.add(base_model)
model.add(keras.layers.AveragePooling2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(500, activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(class_names), activation="softmax"))

model.layers[1].trainable = False

model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=keras.optimizers.Adam(1e-3), 
              metrics=["accuracy"])


model.summary()

In [ ]:
model.fit(train_ds, epochs=1, validation_data=valid_ds)

In [ ]:
model.layers[1].trainable = True
model.summary()

In [ ]:
model.fit(train_ds, epochs=5, validation_data=valid_ds)

Further extensions by augmenting data: https://keras.io/guides/transfer_learning/#using-random-data-augmentation

## Shameless Self Promotion
If you wish to see more of content like this explained buy my [DL course](https://www.udemy.com/course/machine-learning-and-data-science-2021/?referralCode=E79228C7436D74315787) (usually $15).